# Python 知识

# 代码分析 class ColumnGrouper(Configured)

## `__init__`
（1）将 `columns, group_by, allow_enable, allow_disable, allow_modify` 存到 `self._config.items()`
  - 注意 `self._config.__dict__` 中仅有 `readonly = True`

（2）使用 `group_by_to_index` 从 `columns` 选出 `group_by` 对应的，构成新的索引Index存到 `self._group_by`

（3）将 `allow_enable, allow_disable, allow_modify` 分别存到 `self._allow_enable, self._allow_disable, self._allow_modify`
```python
def __init__(self, 
            columns: tp.Index,
            group_by: tp.GroupByLike = None,
            allow_enable: bool = True,
            allow_disable: bool = True,
            allow_modify: bool = True) -> None:
            
    Configured.__init__(
        self,
        columns=columns,
        group_by=group_by,
        allow_enable=allow_enable,
        allow_disable=allow_disable,
        allow_modify=allow_modify
    )

    checks.assert_instance_of(columns, pd.Index)
    self._columns = columns  
    
    if group_by is None or group_by is False:
        self._group_by = None
    else:
        self._group_by = group_by_to_index(columns, group_by)

    self._allow_enable = allow_enable    
    self._allow_disable = allow_disable   
    self._allow_modify = allow_modify    
```

## is_grouped
- 根据参数 `group_by`
  - `False`：表示**强制不分组**，返回 `False`
  - `None`：使用实例的默认分组 `group_by = self.group_by`
- `return group_by is not None`：表示是否有分组


```python
def is_grouped(self, group_by: tp.GroupByLike = None) -> bool:
    if group_by is False:
        return False
    if group_by is None:
        group_by = self.group_by
    return group_by is not None
```

## is_grouping_modified
调用时：
- 在 `self.__dict__` 中查找是否有 `__cached_is_grouping_modified` 对应的属性
    ```python
    def partial_is_grouping_modified(group_by) -> tp.Any:
        return is_grouping_modified(self, group_by)
    ```
    没有则将 `partial_func` 进行**lru_cache**化为 `cached_is_grouping_modified` 作为值
- 如果 `group_by` 是可哈希的，调用 `cached_is_grouping_modified(group_by)`，否则 `is_grouping_modified(self, group_by)`。
  - 使用 `group_by_to_index` 从 `self.columns` 选出 `group_by` 对应的 Index 替换 `group_by`
  - 比较 `group_by` 和 `self.group_by` 是否一致

```python
@cached_method
def is_grouping_modified(self, group_by: tp.GroupByLike = None) -> bool:
    if group_by is None or (group_by is False and self.group_by is None):
        return False
        
    group_by = group_by_to_index(self.columns, group_by)
    
    if isinstance(group_by, pd.Index) and isinstance(self.group_by, pd.Index):
        if not pd.Index.equals(group_by, self.group_by):
            groups1 = get_groups_and_index(self.columns, group_by)[0]
            groups2 = get_groups_and_index(self.columns, self.group_by)[0]
            if not np.array_equal(groups1, groups2):
                return True
        return False
    return True
```